In [1]:
!git clone https://github.com/sk27110/ImageCaption.git

Cloning into 'ImageCaption'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 43 (delta 23), reused 36 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 123.83 KiB | 1.19 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [3]:
%cd ImageCaption/

/content/ImageCaption


In [4]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.3 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.3.2
    Uninstalling python-box-7.3.2:
      Successfully uninstalled python-box-7.3.2


In [5]:
!pip install comet_ml

In [6]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 137.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
from comet_ml import API
from pathlib import Path

api = API(api_key="MbL2psOHT82Uc7ML5Cd7TSvmR")

workspace = "-3767"
project_name = "image_captioning"
experiment_key = "8c81ec6aa5fd437c9213a27a572b7fc3"

exp = api.get_experiment(workspace, project_name, experiment_key)

output_dir = Path("./from_experiment_assets")
output_dir.mkdir(exist_ok=True)

for asset in exp.get_asset_list():
    fname = asset["fileName"]
    if "best_model_latest" in fname:
        binary = exp.get_asset(asset["assetId"], return_type="binary")

        with open(output_dir / fname, "wb") as f:
            f.write(binary)

        print("✅ Скачан:", fname)


✅ Скачан: best_model_latest.pth


In [8]:
!ls

from_experiment_assets	logs.log      main.py	 requirements.txt  test.ipynb
HW5.pdf			lstm_main.py  README.md  src


In [9]:
from src.dataset import get_datasets

In [10]:
train , _, test = get_datasets()

Using Colab cache for faster access to the 'flickr8k' dataset.
Path to dataset files: /kaggle/input/flickr8k


In [11]:
for asset in exp.get_asset_list():
    fname = asset["fileName"]
    # print(fname)
    if "tokenizer" in fname:
        binary = exp.get_asset(asset["assetId"], return_type="binary")

        with open(output_dir / fname, "wb") as f:
            f.write(binary)

        print("✅ Скачан:", fname)

✅ Скачан: tokenizer.pth


In [12]:
from src.model_lstm import LSTMEncoderDecoder
import torch

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("/content/ImageCaption/from_experiment_assets/best_model_latest.pth", map_location=device)
model = LSTMEncoderDecoder(**checkpoint["model_args"]).to(device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 217MB/s]


LSTMEncoderDecoder(
  (encoder): Encoder(
    (cnn): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
   

In [14]:
from src.dataset import Vocabulary

In [15]:
checkpoint = torch.load("/content/ImageCaption/from_experiment_assets/tokenizer.pth", map_location=device)

In [16]:
itos = checkpoint['itos']
stoi = checkpoint['stoi']

In [17]:
input = test[0][0].unsqueeze(0).to(device)

In [18]:
gen_ids=model.generate(input)[0]

In [19]:
gen_tokens = [itos[idx] for idx in gen_ids if idx != stoi["<PAD>"]]

In [20]:
gen_tokens

['<START>',
 'a',
 'group',
 'of',
 'people',
 'are',
 'sitting',
 'on',
 'a',
 '<UNK>',
 '<UNK>',
 'looking',
 'at',
 'a',
 '<UNK>',
 '.',
 '<END>']

In [21]:
!pip install pycocoevalcap
!pip install pycocotools

In [22]:
from tqdm import tqdm
from pycocoevalcap.cider.cider import Cider

In [28]:
pred_caption = dict()
true_captions = dict()

for img, captions, img_name in tqdm(test):
    input = img.unsqueeze(0).to(device)
    gen_ids = model.generate_beam(
                                input,
                                max_len=25,
                                beam_width=3,
                                start_token=stoi["<START>"],
                                end_token=stoi["<END>"]
                            )[0]
    gen_tokens = [itos[idx] for idx in gen_ids if idx != stoi["<PAD>"]]
    clear_tokens = []
    for token in gen_tokens:
        if token == '<START>' or token=='<END>':
          continue
        clear_tokens.append(token)

    pred_caption[img_name] = [" ".join(clear_tokens)]
    true_captions[img_name] = captions


100%|██████████| 810/810 [00:32<00:00, 25.20it/s]


In [25]:
pred_caption

{'1022454428_b6b660a67b.jpg': ['a group of people are sitting on a <UNK> <UNK> .'],
 '102351840_323e3de834.jpg': ['three people are standing on the beach .'],
 '1028205764_7e8df9a2ea.jpg': ['two boys are sitting on a river .'],
 '103195344_5d2dc613a3.jpg': ['a man in a red shirt is standing in front of a building .'],
 '1042590306_95dea0916c.jpg': ['two young girls pose for a picture .'],
 '105342180_4d4a40b47f.jpg': ['a girl in a blue shirt is standing on a lake .'],
 '1056249424_ef2a2e041c.jpg': ['two children playing in the sand .'],
 '1075867198_27ca2e7efe.jpg': ['a man and a man standing on the beach .'],
 '1075881101_d55c46bece.jpg': ['a boy in a blue shirt is sitting in a pool .'],
 '1079274291_9aaf896cc1.jpg': ['two young boys are smiling in the air .'],
 '1093737381_b313cd49ff.jpg': ['a young boy in a red shirt is standing on the street .'],
 '1094462889_f9966dafa6.jpg': ['a dog running through the snow .'],
 '1095980313_3c94799968.jpg': ['a girl in a pink bathing suit is stan

In [27]:
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.spice.spice import Spice
from pycocoevalcap.cider.cider import Cider

scorers = [
    (Bleu(4), ["BLEU-1", "BLEU-2", "BLEU-3", "BLEU-4"]),
    (Meteor(), "METEOR"),
    (Rouge(), "ROUGE-L"),
    (Cider(), "CIDEr"),
]

for scorer, name in scorers:
    score, scores = scorer.compute_score(true_captions, pred_caption)

    if isinstance(name, list):
        print(f"{name[3]}: {score[3]:.3f}")
    else:
        print(f"{name}: {score:.3f}")

{'testlen': 7652, 'reflen': 7886, 'guess': [7652, 6842, 6032, 5222], 'correct': [5066, 2019, 827, 314]}
ratio: 0.9703271620592225
BLEU-4: 0.194
METEOR: 0.216
ROUGE-L: 0.461
CIDEr: 0.516
